# Natural Language Processing
Die Verarbeitung von natürlicher Sprache ist eine der größten Herausforderungen in der Informatik.

### Bibliotheken:
- NLTK
- PyTorch NLP
- SpaCy


### Installation:
Neben der einfachen Installation von nltk über **pip install nltk**, benötigt die Bibliothek häufig sogenannte Corpora. Sollte ein solcher fehlen, wird entsprechend in der Fehlermeldung darauf hingewiesen. mit **nltk.download()** im Terminal wird das Download-Fenster von nltk geöffnet und die fehlenden Dateien nachgeladen. Im Beispiel-Code wird die Bibliothek per Befehl runtergeladen (_nltk.download('punkt')_).

In [1]:
import nltk

# Falls ein Proxy genutzt wird
# nltk.set_proxy('http://proxy.example.com:3128', ('USERNAME', 'PASSWORD'))

# Herunterladen aller notwendigen Corpora
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')

# entity relation
nltk.download('ieer')
nltk.download('conll2002')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package ieer to /root/nltk_data...
[nltk_data]   Package ieer is already up-to-date!
[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


True

## Extraktion von Informationen
Im Folgenden soll der gesamte Prozess zur Extraktion von Informationen aus Texten aufgezeigt und programmatisch abgebildet werden. Dieser ist:

Raw Text -> Tokenisierung (Satzebene) -> Tokenisierung (Wortebene) -> Part Of Speech Tagging -> Entity Detection -> Relation Detection

### Tokenization zum Unterteilen von Texten
Bevor ein Text verarbeitet wird, wird er über einen Tokenizer in kleinere Einheiten unterteilt, etwa Sätze und Wörter.

In [2]:
# German corpus
# https://datascience.blog.wzb.eu/2016/07/13/accurate-part-of-speech-tagging-of-german-texts-with-nltk/

text = "Cyber Security is a challenge for the German government and hence for the German chancellor Angela Merkel."
#text = "Cyber Security is a challenge for the German government which is ran by chancellor Angela Merkel."

# source: https://www.wired.com/story/notpetya-cyberattack-ukraine-russia-code-crashed-the-world/
#text = open("data/notpetya.txt", "r").read()

# Tokenizer zum Unterteilen von Texten in einzelne Sätze
sentences = nltk.tokenize.sent_tokenize(text)
print("Satzebene:")
print(sentences)

# Tokenizer zum Unterteilen von Texten in einzelne Wörter.
tokens = nltk.tokenize.word_tokenize(text)
print("\nWortebene:")
print(tokens)

Satzebene:
['Cyber Security is a challenge for the German government and hence for the German chancellor Angela Merkel.']

Wortebene:
['Cyber', 'Security', 'is', 'a', 'challenge', 'for', 'the', 'German', 'government', 'and', 'hence', 'for', 'the', 'German', 'chancellor', 'Angela', 'Merkel', '.']


### (Optional) Stemming zum Normalisieren von Wörtern
Sollte es nötig sein Wörter vor der Verarbeitung zu normalisieren, trifft man häufig auf den Begriff Stemmer. Dieser wandet u.a. Mehrzahlbegriffe in Einzahlbegriffe.

In [3]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

# porter stemmer
stemmer_porter = PorterStemmer()
normalized_words_porter = [stemmer_porter.stem(token) for token in tokens]

print("Porter result:")
print(normalized_words_porter)

# snowball stemmer
stemmer_snowball = SnowballStemmer("english", ignore_stopwords=True)
normlized_words_snowball = [stemmer_snowball.stem(token) for token in tokens]

print("Snowball result:")
print(normlized_words_snowball)

Porter result:
['cyber', 'secur', 'is', 'a', 'challeng', 'for', 'the', 'german', 'govern', 'and', 'henc', 'for', 'the', 'german', 'chancellor', 'angela', 'merkel', '.']
Snowball result:
['cyber', 'secur', 'is', 'a', 'challeng', 'for', 'the', 'german', 'govern', 'and', 'henc', 'for', 'the', 'german', 'chancellor', 'angela', 'merkel', '.']


### (Optional) Lemmatizer für Wortnormalisierungen
Ein besseres Ergebnis als das Stemming liefert ein Lemmatizer auf Basis einer Wortliste.

In [4]:
from nltk.stem import WordNetLemmatizer
lemmatizer_wordnet = WordNetLemmatizer()
normlized_words_wordnet = [lemmatizer_wordnet.lemmatize(token) for token in tokens]

print("Lemmatizer WordNet results:")
print(normlized_words_wordnet)

Lemmatizer WordNet results:
['Cyber', 'Security', 'is', 'a', 'challenge', 'for', 'the', 'German', 'government', 'and', 'hence', 'for', 'the', 'German', 'chancellor', 'Angela', 'Merkel', '.']


### Part of Speech (POS) Tagging
Beim POS-tagging werden die Klassen einzelner Wörter im Satz erfasst, also z.B. Nomen, Verben und Präpositionen. Die Ausgabe mag etwas kryptisch erscheinen, jedoch  bietet _nltk_ mehrere Funktionen, um mit dem Ergebnis zu arbeiten. Diese sollen im späteren Verlauf betrachtet werden.

Übliche Tags beim Part of Speech Tagging sind hier zu sehen (Quelle: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html)

<table>
	<tr>
		<td><b>Nummer</b></td>
		<td><b>Tag</b></td>
		<td><b>Beschreibung</b></td>
        <td><b>Nummer</b></td>
		<td><b>Tag</b></td>
		<td><b>Beschreibung</b></td>
	</tr>
  <tr> 
    <td> 1. </td>
    <td>CC </td>
    <td>Coordinating conjunction </td>
    <td> 2. </td>
    <td>CD </td>
    <td>Cardinal number </td>
  </tr>
  <tr> 
    <td> 3. </td>
    <td>DT </td>
    <td>Determiner </td>
    <td> 4. </td>
    <td>EX </td>
    <td>Existential <i>there<i> </i></i></td>
  </tr>
  <tr> 
    <td> 5. </td>
    <td>FW </td>
    <td>Foreign word </td>
    <td> 6. </td>
    <td>IN </td>
    <td>Preposition or subordinating conjunction </td>
  </tr>
  <tr> 
    <td> 7. </td>
    <td>JJ </td>
    <td>Adjective </td>
    <td> 8. </td>
    <td>JJR </td>
    <td>Adjective, comparative </td>
  </tr>
  <tr> 
    <td> 9. </td>
    <td>JJS </td>
    <td>Adjective, superlative </td>
    <td> 10. </td>
    <td>LS </td>
    <td>List item marker </td>
  </tr>
  <tr> 
    <td> 11. </td>
    <td>MD </td>
    <td>Modal </td> 
    <td> 12. </td>
    <td>NN </td>
    <td>Noun, singular or mass </td>
  </tr>
  <tr> 
    <td> 13. </td>
    <td>NNS </td>
    <td>Noun, plural </td>
    <td> 14. </td>
    <td>NNP </td>
    <td>Proper noun, singular </td>
  </tr>
  <tr> 
    <td> 15. </td>
    <td>NNPS </td>
    <td>Proper noun, plural </td>
    <td> 16. </td>
    <td>PDT </td>
    <td>Predeterminer </td>
  </tr>
  <tr> 
    <td> 17. </td>
    <td>POS </td>
    <td>Possessive ending </td>
    <td> 18. </td>
    <td>PRP </td>
    <td>Personal pronoun </td>
  </tr>
  <tr> 
    <td> 19. </td>
    <td>PRP&#36; </td>
    <td>Possessive pronoun </td>
    <td> 20. </td>
    <td>RB </td>
    <td>Adverb </td>
  </tr>
  <tr> 
    <td> 21. </td>
    <td>RBR </td>
    <td>Adverb, comparative </td>
    <td> 22. </td>
    <td>RBS </td>
    <td>Adverb, superlative </td>
  </tr>
  <tr> 
    <td> 23. </td>
    <td>RP </td>
    <td>Particle </td>
    <td> 24. </td>
    <td>SYM </td>
    <td>Symbol </td>
  </tr>
  <tr> 
    <td> 25. </td>
    <td>TO </td>
    <td><i>to</i> </td> 
    <td> 26. </td>
    <td>UH </td>
    <td>Interjection </td>
  </tr>
  <tr> 
    <td> 27. </td>
    <td>VB </td>
    <td>Verb, base form </td>
    <td> 28. </td>
    <td>VBD </td>
    <td>Verb, past tense </td>
  </tr>
  <tr> 
    <td> 29. </td>
    <td>VBG </td>
    <td>Verb, gerund or present participle </td>
    <td> 30. </td>
    <td>VBN </td>
    <td>Verb, past participle </td>
  </tr>
  <tr> 
    <td> 31. </td>
    <td>VBP </td>
    <td>Verb, non-3rd person singular present </td>
    <td> 32. </td>
    <td>VBZ </td>
    <td>Verb, 3rd person singular present </td>
  </tr>
  <tr> 
    <td> 33. </td>
    <td>WDT </td>
    <td>Wh-determiner </td>
    <td> 34. </td>
    <td>WP </td>
    <td>Wh-pronoun </td>
  </tr>
  <tr> 
    <td> 35. </td>
    <td>WP$ </td>
    <td>Possessive wh-pronoun </td>
    <td> 36. </td>
    <td>WRB </td>
    <td>Wh-adverb</td>
  </tr>
</table>

In [5]:
tagged_tokens = nltk.pos_tag(tokens)
print(tagged_tokens)

# Zugriff auf Elemente eines einzelnen Tokens
for t in tagged_tokens:
    print("Word: " + t[0] + "\nWortart: " + t[1] + "\n")

[('Cyber', 'NNP'), ('Security', 'NNP'), ('is', 'VBZ'), ('a', 'DT'), ('challenge', 'NN'), ('for', 'IN'), ('the', 'DT'), ('German', 'JJ'), ('government', 'NN'), ('and', 'CC'), ('hence', 'NN'), ('for', 'IN'), ('the', 'DT'), ('German', 'JJ'), ('chancellor', 'NN'), ('Angela', 'NNP'), ('Merkel', 'NNP'), ('.', '.')]
Word: Cyber
Wortart: NNP

Word: Security
Wortart: NNP

Word: is
Wortart: VBZ

Word: a
Wortart: DT

Word: challenge
Wortart: NN

Word: for
Wortart: IN

Word: the
Wortart: DT

Word: German
Wortart: JJ

Word: government
Wortart: NN

Word: and
Wortart: CC

Word: hence
Wortart: NN

Word: for
Wortart: IN

Word: the
Wortart: DT

Word: German
Wortart: JJ

Word: chancellor
Wortart: NN

Word: Angela
Wortart: NNP

Word: Merkel
Wortart: NNP

Word: .
Wortart: .



### Named Entity Recognition (NER) über Chunking
Chunking ist eine Methode, die zum Unterteilen von Sätzen und Erfassen und Klassifizieren von Entitäten aus Texten eingesetzt wird. Diese setzt ein vorheriges POS-Tagging voraus und resultiert in einer Baumstruktur, die Sätze in verschachtelte, logische und nicht überlappende Gruppen untergliedert.

Das vorgehen wird klarer, wenn man ein Beispiel betrachtet. Hervorgehoben werden die Begriffe _Germany_ und _Angela Merkel_, die jeweils als GPE (Geo-Political Entity) und als Person erkannt werden.

Die üblicherweise erkannten Entitätstypen sind:

- ORGANIZATION
- PERSON
- LOCATION
- DATE
- TIME
- MONEY
- PERCENT
- FACILITY
- GPE (Geo-Political Entity)

__TODO__: Im Abschnitt über Machine Learning ist zu sehen, wie ein eigener NER auf Basis von einer vorher getaggten Datensatz trainiert wird.

In [6]:
ner_tree = nltk.ne_chunk(tagged_tokens)
print(ner_tree)

# TODO Ein Baum kann auch als Grafik ausgegeben werden, funktioniert jedoch noch nicht
# in Jupyter Notebook

# Eine Baumstruktur kann mit wenig Aufwand durchlaufen werden
def traverse(t):
    try:
        t.label()
    except AttributeError:
        print(t, end=" ")
    else:
        # Now we know that t.node is defined
        print('(', t.label(), end=" ")
        for child in t:
            traverse(child)
        print(')', end=" ")
        
traverse(ner_tree)

(S
  Cyber/NNP
  Security/NNP
  is/VBZ
  a/DT
  challenge/NN
  for/IN
  the/DT
  (GPE German/JJ)
  government/NN
  and/CC
  hence/NN
  for/IN
  the/DT
  (GPE German/JJ)
  chancellor/NN
  (PERSON Angela/NNP Merkel/NNP)
  ./.)
( S ('Cyber', 'NNP') ('Security', 'NNP') ('is', 'VBZ') ('a', 'DT') ('challenge', 'NN') ('for', 'IN') ('the', 'DT') ( GPE ('German', 'JJ') ) ('government', 'NN') ('and', 'CC') ('hence', 'NN') ('for', 'IN') ('the', 'DT') ( GPE ('German', 'JJ') ) ('chancellor', 'NN') ( PERSON ('Angela', 'NNP') ('Merkel', 'NNP') ) ('.', '.') ) 

### Relation Extraction
Nachdem Entitäten in Texten erkannt wurden, will man in der Regel Beziehungen zwischen ihnen herstellen. Eine Relation kann u.a. durch zwei Entitäten und ein Bindewort hergestellt werden.

__TODO__: Zum Laufen bringen

In [7]:
import re
from nltk.corpus import ieer

AS = re.compile(r'.*\bof\b.*')
for cs in tagged_tokens:
    relations = nltk.sem.relextract.extract_rels('PER', 'ORG', cs, corpus='conll2002')
    for rel in relations:
        print (nltk.sem.relextract.rtuple(rel))

**TODO**: Zeitliche Einordnung von Events

**TODO**: Big Picture in Graph

### NLP mit SpaCy
Nun soll eine weitere Bibliothek betrachtet werden, die in der Lage ist natürliche Sprache zu analysieren. SpaCy ist etwas jünger als NLTK und sieht sich eher für industriellen Einsatz geeignet als NLTK, das für Forschung und Lehre konzipiert wurde.

In [8]:
import spacy

# Installation des Models für deutsche Sprache: python -m spacy download de_core_news_sm
#nlp = spacy.load('de_core_news_sm')
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)

for token in doc:
    print(token.text, token.pos_, token.dep_)

Cyber PROPN compound
Security PROPN nsubj
is VERB ROOT
a DET det
challenge NOUN attr
for ADP prep
the DET det
German ADJ amod
government NOUN pobj
and CCONJ cc
hence ADV advmod
for ADP conj
the DET det
German ADJ amod
chancellor NOUN pobj
Angela PROPN compound
Merkel PROPN appos
. PUNCT punct


In [9]:
# Liste alle Entitäten des Texts auf
for entity in doc.ents:
    print(entity.text, entity.label_)

German NORP
German NORP
Angela Merkel PERSON


In [10]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)

In [11]:
displacy.render(doc, style='ent', jupyter=True)

In [17]:
from spacy import displacy

be_verbs_present = ['am', 'are', 'is']

def extract_relations(doc):
    # Erzeuge eine Liste aus Entities und noun_chunks (Nomen samt Beiwerk)
    # und füge alle Spans zu einem einzelnen Token zusammen.
    spans = list(doc.ents) + list(doc.noun_chunks)
    for span in spans:
        span.merge()
    
    # Die Baumansicht ändert sich durch das Zusammenziehen von zusammegehörigen Nomen
    print("Visualization of joined entities and noun chunks:")
    displacy.render(doc, style='dep', jupyter=True)

    relations = []
    for person in filter(lambda w: w.ent_type_ == 'PERSON', doc):        
        if person.dep_ in ('nsubj'):
            if person.head.pos_ == 'VERB' and person.head.text in be_verbs_present:
                rights = list(person.head.rights)
                for r in rights:
                    if r.dep_ in ('attr'):
                        relations.append((person, r))

    return relations

doc = nlp("Angela Merkel is the chancellor of Germany.")
print("Source text:", doc)
print("Visualization of tokens:")
displacy.render(doc, style='dep', jupyter=True)
relations = extract_relations(doc)
for r1, r2 in relations:
    print('{}\t{}\t{}'.format(r1.text, r1.ent_type_, r2.text))

Source text: Angela Merkel is the chancellor of Germany.
Visualization of tokens:


Visualization of joined entities and noun chunks:


Angela Merkel	PERSON	the chancellor


# Sprachenerkennung
Häufig müssen Sprachen normalisiert werden, bevor sie analysiert werden können, sprich in eine gleiche Sprache übersetzt werden. Da viele Werkzeuge gute Corpora für Englisch bereitstellen, bietet sich die Übersetzung ins Englische an. Um eine Übersetzung durchzuführen, muss jedoch zunächst die Quellsprache ermittelt werden.

Der einfachste Ansatz ist die Bibliothek _langdetect_ zu verwenden.

In [ ]:
from langdetect import detect

print(detect("War doesn't show who's right, just who's left."))

print(detect("Ein, zwei, drei, vier"))

### Spracherkennung anhand von Stopwords

Ein probabilistischer Ansatz berechnet die Wahrscheinlichkeiten, dass ein Text in Spracht a, b oder c geschrieben ist, anhand der Vorkommnisse sogenannter Stopwords. Stopwords sind Begriffe wie "ich", "haben", "auf", "im" ... die für eine Analyse in den meisten Fällen wenig Bedeutung haben und oft herausgefiltert werden. Der Vorteil dieses Vorgehens ist, dass er recht einfach ist und _nltk_ eine eigene Liste von Stopwords pflegt.

Quelle: http://blog.alejandronolla.com/2013/05/15/detecting-text-language-with-python-and-nltk/

In [ ]:
from nltk.corpus import stopwords
from nltk import wordpunct_tokenize

def detect_language(text):
    languages_ratios = {}
    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens]

    # Wieviele Stopwords von Sprache language sind in text
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(words)
        # Bereichne die Schnittmenge von stopwords und text
        common_elements = words_set.intersection(stopwords_set)

        # Schreibe die Anzahl der Elemente der Schnittmenge als Wert für die Sprache language
        languages_ratios[language] = len(common_elements)

    most_rated_language = max(languages_ratios, key=languages_ratios.get)
    return most_rated_language

nltk.download('stopwords')

print(detect_language("Sicherheit ist sehr wichtig für uns."))